In [ ]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import string
import re
import stanza
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import gensim
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import nltk


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-16 12:40:06.673618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 12:40:06.865194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-16 12:40:06.865223: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202

We started by importing various libraries that provide functions and tools for text preprocessing, feature extraction, modeling, and evaluation.

In [ ]:
data = pd.read_csv('/work/-20230526-132402/yelp_review.csv')

In [ ]:
data = data.dropna(subset=['text'])

In [ ]:
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.3 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Visualizing the data:

In [ ]:
data=data[['stars','text']]

In [ ]:
data

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...
...,...,...
9995,3,First visit...Had lunch here today - used my G...
9996,4,Should be called house of deliciousness!\n\nI ...
9997,4,I recently visited Olive and Ivy for business ...
9998,2,My nephew just moved to Scottsdale recently so...


The text column is considered as the main text data for analysis, and the stars column represents the target labels.

## Pre-processing

### Tokenization and lemmatization

In [ ]:
reviews = data['text'].tolist() # we converted the 'text' column to a list of strings
tokenized_reviews = []
lemmatized_reviews = []
batch_size = 1000 # we used a batch size because it was taking too long
# so to improve the efficiency and speed of the processing


In [ ]:
for i in range(0, len(reviews), batch_size):
    batch_reviews = reviews[i:i+batch_size]
    docs = list(nlp.pipe(batch_reviews))

    # we extracted the tokens
    batch_tokens = [[token.text for token in doc] for doc in docs]
    tokenized_reviews.extend(batch_tokens)

    # Doing Lemmatization
    batch_lemmas = [[token.lemma_ for token in doc if token.lemma_ != '-PRON-'] for doc in docs]
    lemmatized_reviews.extend(batch_lemmas)

After doing both, we add them to our dataset.

In [ ]:
data['tokenized_text'] = tokenized_reviews
data['lemmatized_text'] = lemmatized_reviews

# Print the first review and its tokenized version
print('Original Review:')
print(data['text'][0])

print('Tokenized Review:')
print(data['tokenized_text'][0])


print('Lemmatized Review:')
print(data['lemmatized_text'][0])

Original Review:
My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
Tokenized Review:
['My', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakf

Adding additional pre-processing operations: these additional preprocessing steps contribute to cleaner, more meaningful, and consistent text data, which can lead to improved performance in sentiment analysis.

### Handling punctuation

In [ ]:
import string
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

data['tokenized_text'] = data['tokenized_text'].apply(lambda x: [remove_punctuation(token) for token in x])


In [ ]:
print(data['tokenized_text'][0])


['My', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakfast', 'and', 'it', 'was', 'excellent', '', ' ', 'The', 'weather', 'was', 'perfect', 'which', 'made', 'sitting', 'outside', 'overlooking', 'their', 'grounds', 'an', 'absolute', 'pleasure', '', ' ', 'Our', 'waitress', 'was', 'excellent', 'and', 'our', 'food', 'arrived', 'quickly', 'on', 'the', 'semi', '', 'busy', 'Saturday', 'morning', '', ' ', 'It', 'looked', 'like', 'the', 'place', 'fills', 'up', 'pretty', 'quickly', 'so', 'the', 'earlier', 'you', 'get', 'here', 'the', 'better', '', '\n\n', 'Do', 'yourself', 'a', 'favor', 'and', 'get', 'their', 'Bloody', 'Mary', '', ' ', 'It', 'was', 'phenomenal', 'and', 'simply', 'the', 'best', 'I', 've', 'ever', 'had', '', ' ', 'I', 'm', 'pretty', 'sure', 'they', 'only', 'use', 'ingredients', 'from', 'their', 'garden', 'and', 'blend', 'them', 'fresh', 'when', 'you', 'order', 'it', '', ' ', 'It', 'was', 'amazing', '', '\n\n', 'While', 'EVERYTHING', 'on', 'the', 'menu', 'looks',

### Removing stopwords

In [ ]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stopwords]
data['tokenized_text'] = data['tokenized_text'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(data['tokenized_text'][0])

['wife', 'took', 'birthday', 'breakfast', 'excellent', '', ' ', 'weather', 'perfect', 'made', 'sitting', 'outside', 'overlooking', 'grounds', 'absolute', 'pleasure', '', ' ', 'waitress', 'excellent', 'food', 'arrived', 'quickly', 'semi', '', 'busy', 'Saturday', 'morning', '', ' ', 'looked', 'like', 'place', 'fills', 'pretty', 'quickly', 'earlier', 'get', 'better', '', '\n\n', 'favor', 'get', 'Bloody', 'Mary', '', ' ', 'phenomenal', 'simply', 'best', 'ever', '', ' ', 'pretty', 'sure', 'use', 'ingredients', 'garden', 'blend', 'fresh', 'order', '', ' ', 'amazing', '', '\n\n', 'EVERYTHING', 'menu', 'looks', 'excellent', '', 'white', 'truffle', 'scrambled', 'eggs', 'vegetable', 'skillet', 'tasty', 'delicious', '', ' ', 'came', '2', 'pieces', 'griddled', 'bread', 'amazing', 'absolutely', 'made', 'meal', 'complete', '', ' ', 'best', '', 'toast', '', 'ever', '', '\n\n', 'Anyway', '', 'ca', 'nt', 'wait', 'go', 'back', '']


### Lowercasing

In [ ]:
data['tokenized_text'] = data['tokenized_text'].apply(lambda x: [token.lower() for token in x])

In [ ]:
print(data['tokenized_text'][0])

['wife', 'took', 'birthday', 'breakfast', 'excellent', '', ' ', 'weather', 'perfect', 'made', 'sitting', 'outside', 'overlooking', 'grounds', 'absolute', 'pleasure', '', ' ', 'waitress', 'excellent', 'food', 'arrived', 'quickly', 'semi', '', 'busy', 'saturday', 'morning', '', ' ', 'looked', 'like', 'place', 'fills', 'pretty', 'quickly', 'earlier', 'get', 'better', '', '\n\n', 'favor', 'get', 'bloody', 'mary', '', ' ', 'phenomenal', 'simply', 'best', 'ever', '', ' ', 'pretty', 'sure', 'use', 'ingredients', 'garden', 'blend', 'fresh', 'order', '', ' ', 'amazing', '', '\n\n', 'everything', 'menu', 'looks', 'excellent', '', 'white', 'truffle', 'scrambled', 'eggs', 'vegetable', 'skillet', 'tasty', 'delicious', '', ' ', 'came', '2', 'pieces', 'griddled', 'bread', 'amazing', 'absolutely', 'made', 'meal', 'complete', '', ' ', 'best', '', 'toast', '', 'ever', '', '\n\n', 'anyway', '', 'ca', 'nt', 'wait', 'go', 'back', '']


### Handling negations

In [ ]:
def handle_negations(tokens):
    negations = ["not", "no", "n't"]
    handled_tokens = []
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token in negations and i < len(tokens) - 1:
            handled_tokens.append(token + "_" + tokens[i + 1])
            i += 2
        else:
            handled_tokens.append(token)
            i += 1
    return handled_tokens

data['tokenized_text'] = data['tokenized_text'].apply(handle_negations)

In [ ]:
print(data['tokenized_text'][0])

['wife', 'took', 'birthday', 'breakfast', 'excellent', '', ' ', 'weather', 'perfect', 'made', 'sitting', 'outside', 'overlooking', 'grounds', 'absolute', 'pleasure', '', ' ', 'waitress', 'excellent', 'food', 'arrived', 'quickly', 'semi', '', 'busy', 'saturday', 'morning', '', ' ', 'looked', 'like', 'place', 'fills', 'pretty', 'quickly', 'earlier', 'get', 'better', '', '\n\n', 'favor', 'get', 'bloody', 'mary', '', ' ', 'phenomenal', 'simply', 'best', 'ever', '', ' ', 'pretty', 'sure', 'use', 'ingredients', 'garden', 'blend', 'fresh', 'order', '', ' ', 'amazing', '', '\n\n', 'everything', 'menu', 'looks', 'excellent', '', 'white', 'truffle', 'scrambled', 'eggs', 'vegetable', 'skillet', 'tasty', 'delicious', '', ' ', 'came', '2', 'pieces', 'griddled', 'bread', 'amazing', 'absolutely', 'made', 'meal', 'complete', '', ' ', 'best', '', 'toast', '', 'ever', '', '\n\n', 'anyway', '', 'ca', 'nt', 'wait', 'go', 'back', '']


### Removing numbers

In [ ]:
def remove_numbers(tokens):
    return [re.sub(r'\d+', '', token) for token in tokens]

data['tokenized_text'] = data['tokenized_text'].apply(remove_numbers)


Now let's visualise the difference between our original review and the preprocessed review:

In [ ]:
print('Original Review:')
print(data['text'][0])

print('Preprocessed Review:')
print(data['tokenized_text'][0])

Original Review:
My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
Preprocessed Review:
['wife', 'took', 'birthday', 'breakfast', 'excellent', '', ' ', 'weather

## Extracting linguistic features

In [ ]:
from collections import Counter
def extract_linguistic_features(tokens):
    features = Counter(tokens)
    return features

data['linguistic_features'] = data['tokenized_text'].apply(extract_linguistic_features)
#counting the frequency of each unique token in the list

In [ ]:
print('Original Review:')
print(data['text'][0])

print('Tokenized Review:')
print(data['tokenized_text'][0])

print('Linguistic Features:')
print(data['linguistic_features'][0])

Original Review:
My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
Tokenized Review:
['wife', 'took', 'birthday', 'breakfast', 'excellent', '', ' ', 'weather', 

In [ ]:
def extract_features(doc):
    features = []
    for token in doc:
        features.append({
            'token': token.text,
            'lemma': token.lemma_,
            'pos': token.pos_,
            'tag': token.tag_,
            'dep': token.dep_,
            'is_stop': token.is_stop,
            'is_alpha': token.is_alpha,
        })
    return features

data['linguistic_features'] = data['tokenized_text'].apply(lambda x: extract_features(nlp(" ".join(x))))


In [ ]:
print('Linguistic Features:')
print(data['linguistic_features'][0])

Linguistic Features:
[{'token': 'wife', 'lemma': 'wife', 'pos': 'NOUN', 'tag': 'NN', 'dep': 'nsubj', 'is_stop': False, 'is_alpha': True}, {'token': 'took', 'lemma': 'take', 'pos': 'VERB', 'tag': 'VBD', 'dep': 'ROOT', 'is_stop': False, 'is_alpha': True}, {'token': 'birthday', 'lemma': 'birthday', 'pos': 'NOUN', 'tag': 'NN', 'dep': 'compound', 'is_stop': False, 'is_alpha': True}, {'token': 'breakfast', 'lemma': 'breakfast', 'pos': 'NOUN', 'tag': 'NN', 'dep': 'compound', 'is_stop': False, 'is_alpha': True}, {'token': 'excellent', 'lemma': 'excellent', 'pos': 'ADJ', 'tag': 'JJ', 'dep': 'amod', 'is_stop': False, 'is_alpha': True}, {'token': '   ', 'lemma': '   ', 'pos': 'SPACE', 'tag': '_SP', 'dep': 'dep', 'is_stop': False, 'is_alpha': False}, {'token': 'weather', 'lemma': 'weather', 'pos': 'NOUN', 'tag': 'NN', 'dep': 'compound', 'is_stop': False, 'is_alpha': True}, {'token': 'perfect', 'lemma': 'perfect', 'pos': 'ADV', 'tag': 'RB', 'dep': 'dobj', 'is_stop': False, 'is_alpha': True}, {'toke

In [ ]:
data

,stars,text,tokenized_text,lemmatized_text,linguistic_features
0,5,My wife took me here on my birthday for breakf...,"[wife, took, birthday, breakfast, excellent, ,...","[my, wife, take, I, here, on, my, birthday, fo...","[{'token': 'wife', 'lemma': 'wife', 'pos': 'NO..."
1,5,I have no idea why some people give bad review...,"[idea, people, give, bad, reviews, place, , go...","[I, have, no, idea, why, some, people, give, b...","[{'token': 'idea', 'lemma': 'idea', 'pos': 'NO..."
2,4,love the gyro plate. Rice is so good and I als...,"[love, gyro, plate, , rice, good, also, dig, c...","[love, the, gyro, plate, ., rice, be, so, good...","[{'token': 'love', 'lemma': 'love', 'pos': 'NO..."
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...","[rosie, , dakota, , love, chaparral, dog, park...","[Rosie, ,, Dakota, ,, and, I, love, Chaparral,...","[{'token': 'rosie', 'lemma': 'rosie', 'pos': '..."
4,5,General Manager Scott Petello is a good egg!!!...,"[general, manager, scott, petello, good, egg, ...","[General, Manager, Scott, Petello, be, a, good...","[{'token': 'general', 'lemma': 'general', 'pos..."
...,...,...,...,...,...
9995,3,First visit...Had lunch here today - used my G...,"[first, visit, , lunch, today, , used, groupon...","[first, visit, ..., have, lunch, here, today, ...","[{'token': 'first', 'lemma': 'first', 'pos': '..."
9996,4,Should be called house of deliciousness!\n\nI ...,"[called, house, deliciousness, , \n\n, could, ...","[should, be, call, house, of, deliciousness, !...","[{'token': 'called', 'lemma': 'call', 'pos': '..."
9997,4,I recently visited Olive and Ivy for business ...,"[recently, visited, olive, ivy, business, last...","[I, recently, visit, Olive, and, Ivy, for, bus...","[{'token': 'recently', 'lemma': 'recently', 'p..."
9998,2,My nephew just moved to Scottsdale recently so...,"[nephew, moved, scottsdale, recently, bunch, f...","[my, nephew, just, move, to, Scottsdale, recen...","[{'token': 'nephew', 'lemma': 'nephew', 'pos':..."


In [ ]:
corpus = data['tokenized_text']
corpus

0       [wife, took, birthday, breakfast, excellent, ,...
1       [idea, people, give, bad, reviews, place, , go...
2       [love, gyro, plate, , rice, good, also, dig, c...
3       [rosie, , dakota, , love, chaparral, dog, park...
4       [general, manager, scott, petello, good, egg, ...
                              ...                        
9995    [first, visit, , lunch, today, , used, groupon...
9996    [called, house, deliciousness, , \n\n, could, ...
9997    [recently, visited, olive, ivy, business, last...
9998    [nephew, moved, scottsdale, recently, bunch, f...
9999    [, , , locations, , , star, average, , think, ...
Name: tokenized_text, Length: 10000, dtype: object

## GloVe

### Text representation

In [ ]:
import numpy as np
from gensim.models import KeyedVectors

# first thing is to load the pre-trained GloVe word vectors
glove_file = 'glove.6B.100d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = []
for review in corpus:
    embeddings = [word_vectors[word] for word in review if word in word_vectors]
    # Calculating the average embedding for each review
    average_embedding = np.mean(embeddings, axis=0) if embeddings else np.zeros(word_vectors.vector_size)
    X.append(average_embedding)
# We converted the preprocessed reviews in our corpus to their GloVe embeddings



### Training the model

We will train the model using Logistic Regression.

In [ ]:
labels = data['stars'].tolist()
sentiment_labels = []
for rating in labels:
    if rating >= 4:
        sentiment_labels.append('good')
    else:
        sentiment_labels.append('bad')

X = np.array(X)
y = np.array(sentiment_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Let's perform a grid search to help find the best combination of hyperparameters for our model, and get an improved performance and generalization.

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0],
    'max_iter': [100, 200, 300]
}

In [ ]:
logistic_regression = LogisticRegression()

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(logistic_regression, param_grid)
grid_search.fit(X_train, y_train)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1.0, 10.0], 'max_iter': [100, 200, 300],
                         'penalty': ['l1', 'l2']})

In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
predictions_glove = best_model.predict(X_test)

print("Best Hyperparameters:", best_params)
print("Classification Report (GLOVE):")
print(classification_report(y_test, predictions_glove))

Best Hyperparameters: {'C': 1.0, 'max_iter': 100, 'penalty': 'l2'}
Classification Report (GLOVE):
              precision    recall  f1-score   support

         bad       0.69      0.51      0.59       597
        good       0.81      0.90      0.86      1403

    accuracy                           0.79      2000
   macro avg       0.75      0.71      0.72      2000
weighted avg       0.78      0.79      0.78      2000



## Tf-idf

### Text representation

In [ ]:
corpus = [' '.join(tokens) for tokens in data['tokenized_text']]
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 3),
    stop_words='english'
)
tfidf_matrix = vectorizer.fit_transform(corpus)

In [ ]:
print("TF-IDF representation:")
print(tfidf_matrix)


TF-IDF representation:
  (0, 1082)	0.10430864505924704
  (0, 5901)	0.09852408802239128
  (0, 1060)	0.06784855953946042
  (0, 7753)	0.15028839258611135
  (0, 8679)	0.1603994316056082
  (0, 6855)	0.13338725985168615
  (0, 8044)	0.158613000264206
  (0, 784)	0.14273863191166072
  (0, 6850)	0.1491783379936957
  (0, 4781)	0.11151041420382841
  (0, 4952)	0.11635221552711963
  (0, 7655)	0.14616842078383438
  (0, 3000)	0.14616842078383438
  (0, 2631)	0.14354387271617666
  (0, 8067)	0.14712352363139186
  (0, 9556)	0.06954169339804077
  (0, 5734)	0.035580813949284334
  (0, 1059)	0.06720058437798487
  (0, 9121)	0.10953792155862976
  (0, 1662)	0.11508185107262903
  (0, 5241)	0.07146056360170919
  (0, 11)	0.09259478358382288
  (0, 884)	0.07956558620547736
  (0, 6443)	0.10290025242690515
  (0, 1114)	0.0671607569964568
  :	:
  (9999, 5397)	0.1331012336142193
  (9999, 413)	0.13372430679470323
  (9999, 1437)	0.13035751726508896
  (9999, 6035)	0.13063944685200377
  (9999, 2764)	0.12019410721797091
  (999

### Training the model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

labels = data['stars'].tolist()
sentiment_labels = []
for rating in labels:
    if rating >= 4:
        sentiment_labels.append('good')
    else:
        sentiment_labels.append('bad')

X_tfidf = tfidf_matrix.toarray()
y_tfidf = np.array(sentiment_labels)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_tfidf, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100, multi_class='auto')



In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bad       0.83      0.54      0.65       597
        good       0.83      0.95      0.89      1403

    accuracy                           0.83      2000
   macro avg       0.83      0.75      0.77      2000
weighted avg       0.83      0.83      0.82      2000



## Word2Vec

### Text representation

In [ ]:
sentences = data['tokenized_text'].tolist()
word2vec_model = Word2Vec(
    sentences,
    vector_size=300,
    window=5,
    min_count=8,  # higher minimum count to filter out rare words
    workers=4
)

### Training the model

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

In [ ]:
X_w2v = np.array([np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0) for tokens in data['tokenized_text']])
labels = data['stars'].tolist()

sentiment_labels = []
for rating in labels:
    if rating >= 4:
        sentiment_labels.append('good')
    else:
        sentiment_labels.append('bad')

y = np.array(sentiment_labels)

In [ ]:
model_w2v = LogisticRegression()

In [ ]:
grid_search = GridSearchCV(model_w2v, param_grid, cv=5)
grid_search.fit(X_w2v, y)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']})

In [ ]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, random_state=42)

In [ ]:
# Fitting the best model on the training data
best_model.fit(X_train_w2v, y_train_w2v)

# Evaluating the best model on the test set
predictions_w2v = best_model.predict(X_test_w2v)

print("Best Hyperparameters:", best_params)
print("Classification Report (Word2Vec):")
print(classification_report(y_test_w2v, predictions_w2v))

Best Hyperparameters: {'C': 10, 'penalty': 'l2'}
Classification Report (Word2Vec):
              precision    recall  f1-score   support

         bad       0.68      0.50      0.57       597
        good       0.81      0.90      0.85      1403

    accuracy                           0.78      2000
   macro avg       0.74      0.70      0.71      2000
weighted avg       0.77      0.78      0.77      2000

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7159b134-6dce-4d72-befb-c34071b2c35b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>